<h1>MyFitnessPal Data Report</h1>


    
<h2>Data Collection</h2>

In [ ]:
import myfitnesspal as mfp
import keyring
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date, timedelta, datetime
import ast
import re
from termcolor import cprint
import sys



import smtplib

# Import the email modules we'll need
from email.message import EmailMessage

import myfitnesspal as mfp
import keyring
from IPython.display import display, HTML

#PPT Library
from pptx import Presentation
from pptx.util import Inches

client = mfp.Client(USERNAME, PASSWORD)

TDEE = 'Find Your TDEE Here: https://tdeecalculator.net/'
Weight = 'How much do you weigh?'

TDEE_7 = TDEE * 7
weight_7 = 200 * 7 


In [ ]:
#Data For the past Seven Days

t = datetime.today()


#figure out how to simplefy this
thistuple = (t,
             t - timedelta(days=1),
             t - timedelta(days=2), 
             t - timedelta(days=3),
             t - timedelta(days=4),
             t - timedelta(days=5),
             t - timedelta(days=6))


output = []
# output2 = []
for x in thistuple:
    string = str(client.get_date(x))
    date = string[0:8]
    date = [date]
#     output1.append(date)
#grabs the macro nutrient breakdown for that desired date
    nutrition = string[10:-1]
    line2 = nutrition
    temp1 = re.findall(r'\d+', line2) # through regular expression
    nutrition = list(map(int, temp1))
    complete_list = date + nutrition
    output.append(complete_list)
    
multi = pd.DataFrame(output, columns=['Date', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar'])
multi = multi.fillna(0)
multi

In [ ]:
#Data For this week of the month
thistuple2 = (t,
             t - timedelta(days=1),
             t - timedelta(days=2), 
             t - timedelta(days=3),
             t - timedelta(days=4),
             t - timedelta(days=5),
             t - timedelta(days=6))


output = []
# output2 = []
for x in thistuple2:
    string = str(client.get_date(x))
    date = string[0:8]
    date = [date]
#     output1.append(date)
#grabs the macro nutrient breakdown for that desired date
    nutrition = string[10:-1]
    line2 = nutrition
    temp1 = re.findall(r'\d+', line2) # through regular expression
    nutrition = list(map(int, temp1))
    complete_list = date + nutrition
    output.append(complete_list)
        
multi2 = pd.DataFrame(output, columns=['Date', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar'])

#
multi2['Date'] = pd.to_datetime(multi2['Date'])    
multi2['Week'] = multi2.Date.apply(lambda x: x.strftime('%W'))
dftemp = multi2['Week'].iloc[0]
date_this_week = multi2[multi2['Week'] == dftemp]    


date_this_week = date_this_week.fillna(0)
date_this_week

<h3>Generating Charts</h3>

In [ ]:
#Generating Charts

import plotly
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
# import plotly.plotly as py
import plotly.express as px
from IPython.display import Image

# import plotly.io as pio
# pio.orca.config.use_xvfb = True
# animals=['giraffes', 'orangutans', 'monkeys']

# fig = go.Figure([go.Bar(x=animals, y=[20, 14, 23])])

# TDEE_7 = 2900 * 7
Total_Calories_7 = round(multi.Calories.sum())
calories_remaining = round(TDEE_7 - multi.Calories.sum())
caloric_surplus = round(multi.Calories.sum() - TDEE_7)

In [ ]:
#Chart 1: TITLE
x = ['TDEE_7','Total_Calories_7']

if TDEE_7 > multi.Calories.sum():

    fig = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
              hovertext=['Calories Consumed'],
              text=[Total_Calories_7],
              textposition='inside'),
        go.Bar(name='Calories Remaining', x=x, y=[calories_remaining],
              marker_color='whitesmoke',
              hovertext=['Calories Under Weekly TDEE'],
              text=[calories_remaining],
              textposition='inside'
              )
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack')


    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

    fig.update_layout(
        title = 'Caloric Goals For The Past 7 Days'
        )

    

#     fig.show()
    fig.write_image("Images/7DayRollingCalories.png")
    i1 = Image("Images/7DayRollingCalories.png")

#     display(i1)
    


# ^ Good!  
        
elif TDEE_7 < multi.Calories.sum():
        
    fig = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
              marker_color='blue',
              hovertext=['Calories Within Limit'],
              text=[Total_Calories_7],
              textposition='auto'),
        go.Bar(name='Calories Over Maintenance', x=x, y=[caloric_surplus],
              marker_color='red',
              hovertext=['Calories Over Limit'],
              text=[caloric_surplus],
              textposition='auto')
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack')


    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

    fig.update_layout(
        title = 'Caloric Goals For The Past 7 Days'
        )

#     fig.show()
    fig.write_image("Images/7DayRollingCalories.png")
    i1 = Image("Images/7DayRollingCalories.png")

#     display(i1)

else: print ('You Are At a perfect maintenance')

In [ ]:
#Chart 2: Sugar

sugar = multi.Sugar.sum()
carbs = multi.Carbohydrates.sum()    
Carbo_surplus = carbs - sugar

fig3 = go.Figure(data=[
    go.Bar(name='Grams of Sugar Consumed This Week', x=x, y=[sugar],
                          hovertext=['Calories Consumed'],
                          text=[sugar],
                          textposition='inside',
                          marker_color='red'),
    go.Bar(name='Carbohydrates Consumed This Week', x=x, y=[Carbo_surplus],
              marker_color='whitesmoke',
              hovertext=['Calories Under Weekly TDEE'],
              text=[Carbo_surplus])
])
    # Change the bar mode
fig3.update_layout(barmode='stack')


fig3.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),)

fig3.update_layout(
        title = 'How Much of Your Carbs Came From Sugar'
        )

# fig3.show()
fig3.write_image("Images/CarbsFromSugar.png")
i2 = Image("Images/CarbsFromSugar.png")

In [ ]:
#Chart 3 Data
blah = date_this_week.groupby(['Week']).agg({'Protein': pd.Series.sum,
                             'Fat': pd.Series.sum,
                             'Carbohydrates': pd.Series.sum}).reset_index()

blah['Protein'] = blah['Protein'] * 4
blah['Fat'] = blah['Fat'] * 9
blah['Carbohydrates'] = blah['Carbohydrates'] * 4
blah  
#Chart 3

listt = [multi.Protein.sum() * 4, multi.Carbohydrates.sum() * 4, multi.Fat.sum() * 9]

colors = ['Green', 'Blue', 'darkorange']

fig2 = go.Figure(data=[go.Pie(labels=['Protein','Carbohydrates','Fat'], 
                             values=listt)])
fig2.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))


fig2.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),)


fig2.update_layout(
    title_text="Macronutrition Breakdown for the Past 7 Days")
#     # Add annotations in the center of the donut pies.
#     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
#                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])

# fig2.show()
fig2.write_image("Images/7RollingDaysMacros.png")
i3 = Image("Images/7RollingDaysMacros.png")

In [ ]:
#chart 4
Total_Calories_7 = date_this_week.Calories.sum()
calories_remaining = TDEE_7 - date_this_week.Calories.sum()
caloric_surplus = date_this_week.Calories.sum() - TDEE_7


x = ['TDEE_7','Total_Calories_7']

if TDEE_7 > Total_Calories_7:

    fig4 = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
                 hovertext=['Calories Consumed'],
              text=[Total_Calories_7],
              textposition='inside'),
        go.Bar(name='Calories Remaining', x=x, y=[calories_remaining],
                marker_color='whitesmoke',
                hovertext=['Calories Under Weekly TDEE'],
                text=[calories_remaining],
                textposition='inside')
        ])
        # Change the bar mode
    fig4.update_layout(barmode='stack')


    fig4.update_layout(
            autosize=False,
            width=500,
            height=500,
            margin=go.layout.Margin(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),)

    fig4.update_layout(
            title = 'Caloric Goals This Week'
            )

#     fig4.show()
    fig4.write_image("Images/ThisWeekCalories.png")
    i4 = Image("Images/ThisWeekCalories.png")
    

#     display(i4)



elif TDEE_7 < Total_Calories_7:
        
    fig4 = go.Figure(data=[
        go.Bar(name='Calories Consumed This Week', x=x, y=[Total_Calories_7],
              marker_color='blue',
              hovertext=['Calories Within Limit'],
              text=[Total_Calories_7],
              textposition='inside'),
        go.Bar(name='Calories Over Maintenance', x=x, y=[caloric_surplus],
              marker_color='red',
              hovertext=['Calories Over Limit'],
              text=[caloric_surplus],
              textposition='inside')
    ])
    # Change the bar mode
    fig4.update_layout(barmode='stack')


    fig4.update_layout(
        autosize=False,
        width=500,
        height=500,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),)

    fig4.update_layout(
        title = 'Caloric Goals This Week'
        )

#     fig4.show()
    fig4.write_image("Images/ThisWeekCalories.png")
    i4 = Image("Images/ThisWeekCalories.png")


In [ ]:
#Chart 5

listt = [date_this_week.Protein.sum() * 4, date_this_week.Carbohydrates.sum() * 4, date_this_week.Fat.sum() * 9]

colors = ['Green', 'Blue', 'darkorange']

fig5 = go.Figure(data=[go.Pie(labels=['Protein','Carbohydrates','Fat'], 
                             values=listt)])
fig5.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))


fig5.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),)


fig5.update_layout(
    title_text="Macronutrition Breakdown for this Week Thus Far")
#     # Add annotations in the center of the donut pies.
#     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
#                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])

# fig5.show()
fig5.write_image("Images/ThisWeekMacros.png")
i5 = Image("Images/ThisWeekMacros.png")


<h2>Creating Text Files</h2>

In [ ]:
f = open("Intro.txt", "w")
f.write("Your Nutritional Breakdown For " + str(datetime.today().date()))
f.write('\n')
f.write('Weekly Caloric Goal For Maintenance:')
f.write(' ')

f.write(str(TDEE_7))
f.write('\n')

f.close()

# f = open("Intro.txt", "r")
# print(f.read())

In [ ]:
f = open("Intro.txt", "w")
# f.write("Your Nutritional Breakdown For " + str(datetime.today().date()))
# f.write('\n')
f.write('Your daily caloric needs for maintenaning current weight:')
f.write(' ')

f.write(str(TDEE_7 / 7))
f.write('\n')

# f.write('To lose a pound a week, you should consume ' + str(int(TDEE_7-3500) / 7))


f.close()

# f = open("Intro.txt", "r")
# print(f.read())

In [ ]:
f = open("Slide_1.txt", "w")
if TDEE_7 > date_this_week.Calories.sum():
    f.write('You are ' + str(TDEE_7 - date_this_week.Calories.sum()) + ' calories under maintenance')
elif TDEE_7 < date_this_week.Calories.sum():
    f.write('You are ' + str(TDEE_7 - date_this_week.Calories.sum()) + ' calories over maintenance')
else:
    f.write('You are at maintenance')
f.write('\n')
f.write ('\n')
f.write("Your caloric intake relative to maintenance:")
f.write('\n')
f.write(str(round(date_this_week.Calories.sum() / (TDEE_7),2)*100))
f.write('\n')

f.write('\n')
f.write('Percent of calories from protein:')
f.write('\n')
if date_this_week.Protein.sum() == 0 and date_this_week.Calories.sum() == 0:
    f.write(str(0))
else:
    f.write(str(int(round((date_this_week.Protein.sum() * 4) / (date_this_week.Calories.sum()),2)*100)))
f.write('\n')
f.write('\n')

f.write('Percent of calories from fat:')
f.write('\n')

if date_this_week.Fat.sum() == 0 and date_this_week.Calories.sum() == 0:
    f.write(str(0))
else:
    f.write(str(int(round((date_this_week.Fat.sum() * 9) / (date_this_week.Calories.sum()),2)*100)))
    
f.write('\n')

f.write('\n')
f.write('Percent of calories from carbohydrates:')
f.write('\n')

if date_this_week.Carbohydrates.sum() == 0 and date_this_week.Calories.sum() == 0:
    f.write(str(0))
else:
    f.write(str(int(round((date_this_week.Carbohydrates.sum() * 4) / (date_this_week.Calories.sum()),2)*100)))

f.write('\n')
f.write('\n')

f.write('You have consumed an average of ' + str(date_this_week.Protein.mean()) + ' grams of protein this week, which is ' + str((round(date_this_week.Protein.mean() / (weight),2)))
+' gram(s) of protein per pound')


f.close()

#open and read the file after the appending:
# f = open("Slide_1.txt", "r")
# print(f.read())

<h2>Creating the PPTX and adding Content</h2>

In [ ]:
# First Slide

from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

prs = Presentation()
title_slide_layout = prs.slide_layouts[0]

slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text = 'Macronutritional Profile For ' + str(datetime.today().date())
subtitle.text = 'For <YOUR NAME>'
# prs.save('Title.pptx')

In [ ]:
# Second Slide

f = open("Intro.txt", "r")

# prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]
slide2 = prs.slides.add_slide(blank_slide_layout)
left = top = width = height = Inches(1)
txBox = slide2.shapes.add_textbox(Inches(0.5), Inches(0.5),Inches(9), Inches(3.0))
# shape.paragraphs[0].alignment = PP_ALIGN.CENTER

tf = txBox.text_frame
tf.word_wrap = True
# tf.text = "Macronutritional breakdown for " + str(date_this_week.Date[0].date()) + str(' and ') + str(date_this_week.Date.iloc[-1].date())

p = tf.add_paragraph()
p = tf.add_paragraph()
p = tf.add_paragraph()

# p = tf.add_paragraph()
p.text = f.read()
p.font.size = Pt(40)
p.alignment = PP_ALIGN.CENTER

p2 = tf.add_paragraph()

p3 = tf.add_paragraph()
p3.text = 'To lose a pound a week, you should consume ' + str(int(TDEE_7-3500) / 7) + ' calories a day.'
p3.font.size = Pt(40)
p3.alignment = PP_ALIGN.CENTER


# prs.save('Caloric_needs.pptx')


In [ ]:
#Third Slide

f = open("Slide_1.txt", "r")

# prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]
slide3 = prs.slides.add_slide(blank_slide_layout)
left = top = width = height = Inches(1)
txBox = slide3.shapes.add_textbox(Inches(0.5), Inches(0.5),Inches(10), Inches(3.0))
tf = txBox.text_frame
tf.word_wrap = True
tf.text = "Macronutritional breakdown for " + str(date_this_week.Date[0].date()) + str(' and ') + str(date_this_week.Date.iloc[-1].date())

p = tf.add_paragraph()
p = tf.add_paragraph()
p.text = f.read()
p.font.size = Pt(20)


In [ ]:
#Fourth Slide
img_path = 'Images/ThisWeekCalories.png'

# prs = Presentation()
slide5 = prs.slides.add_slide(prs.slide_layouts[8])
placeholder2 = slide5.placeholders[1]
picture = placeholder2.insert_picture(img_path)

title = slide5.shapes.title
subtitle = slide5.placeholders[2]
title.text = "Calories you have already consumed this week"


if TDEE_7 > date_this_week.Calories.sum():
    subtitle.text = 'You are ' + str(TDEE_7 - date_this_week.Calories.sum()) + ' calories under maintenance'
elif TDEE_7 < date_this_week.Calories.sum():
    subtitle.text = 'You are ' + str(TDEE_7 - date_this_week.Calories.sum()) + ' calories over maintenance'
else:
    subtitle.text = 'You are at maintenance'




# subtitle.text = "You have recieved X grams of protein, Y grams of fat, and z grams of carbs!"
# prs.save('progress.pptx')

In [ ]:
#Fith Slide
img_path = 'Images/ThisWeekMacros.png'

# prs = Presentation()
slide4 = prs.slides.add_slide(prs.slide_layouts[8])
placeholder1 = slide4.placeholders[1]
picture = placeholder1.insert_picture(img_path)

title = slide4.shapes.title
subtitle = slide4.placeholders[2]
title.text = "Your Macros This Week"
# subtitle.text = "You have recieved X grams of protein, Y grams of fat, and z grams of carbs!"

# prs.save('progress.pptx')

In [ ]:
#Sixth Slide
img_path = 'Images/CarbsFromSugar.png'

slide6 = prs.slides.add_slide(prs.slide_layouts[8])
placeholder2 = slide6.placeholders[1]
picture = placeholder2.insert_picture(img_path)

title = slide6.shapes.title
subtitle = slide6.placeholders[2]
title.text = "Sugar is one of the leading causes of diabetes and Obesity. Limit consumption."
subtitle.text = str(int(round(multi.Sugar.sum() / (multi.Carbohydrates.sum()),2)*100)) + "% of Your total Carbohydrate consumption in the past 7 days has been from sugar."
# prs.save('progress.pptx')


In [ ]:
#Eigth Slide
img_path = 'Images/7DayRollingCalories.png'

slide8 = prs.slides.add_slide(prs.slide_layouts[8])
placeholder2 = slide8.placeholders[1]
picture = placeholder2.insert_picture(img_path)

title = slide8.shapes.title
subtitle = slide8.placeholders[2]
title.text = "Total Calories Consumed Relative to TDEE Over the Past 7 Days."
# subtitle.text = str(int(round(multi.Sugar.sum() / (multi.Carbohydrates.sum()),2)*100)) + "% of Your total Carbohydrate consumption in the past 7 days has been from sugar."
# prs.save('progress.pptx')

In [ ]:
#Ninth Slide
img_path = 'Images/7RollingDaysMacros.png'

slide9 = prs.slides.add_slide(prs.slide_layouts[8])
placeholder2 = slide9.placeholders[1]
picture = placeholder2.insert_picture(img_path)

title = slide9.shapes.title
subtitle = slide9.placeholders[2]
title.text = "Macronutritional breakdown over the past 7 days."
# subtitle.text = str(int(round(multi.Sugar.sum() / (multi.Carbohydrates.sum()),2)*100)) + "% of Your total Carbohydrate consumption in the past 7 days has been from sugar."
prs.save('MFP_Report.pptx')

<h2>Emailing</h2>

In [ ]:
import smtplib, smtplib, ssl
from datetime import date, timedelta, datetime
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
# The Email Address You will be sending this report from
EMAIL_ADDRESS = YOUR_EMAIL
PASSWORD = EMAIL_PASSWORD
receiver_email = RECEIVING_EMAIL


In [ ]:
all
subject = "Nutritional Report For " + str(datetime.today().date()) 
body = "See how you are doing"
sender_email = EMAIL_ADDRESS
#Email address that will recieve the PPTX
password = PASSWORD

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "plain"))

filename = "MFP_Report.pptx"  # In same directory as script

# Open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)